In [428]:
import pandas as pd
import os
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from termcolor import colored
import torch 
import torch.nn as nn
import torch.optim as optim
from torchtext.vocab import FastText, vocab
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.utils.clip_grad import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from gensim.models.phrases import Phrases, Phraser
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TreebankWordTokenizer
from string import punctuation
import re, random, pathlib
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()
import random

from transformers import RobertaModel, RobertaTokenizer, get_linear_schedule_with_warmup, AutoTokenizer, AutoModel

In [5]:
# nltk requirements for words 
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('vader_lexicon')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/kevin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kevin/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/kevin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/kevin/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kevin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [429]:
df = pd.read_csv("../../../Documents/Github/CS399_Project/data/glassdoor_reviews.csv")

In [430]:
columns = ["pros", "cons", "headline"]
df = df.dropna(subset=['work_life_balance'])

In [431]:
#types_str = df.select_dtypes(include='object').columns
df[columns] = df[columns].fillna("NA")

In [432]:
df[columns] = df[columns].astype(str)

In [433]:
# Preprocessing text routines 
stemmer = WordNetLemmatizer()
tok = TreebankWordTokenizer()

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english')) # common english stop words
to_be_removed = list(en_stop) + list(punctuation)

# Preprocess text for transformers
def preprocess_transformers(string, full_process=True):
        # Remove all the special characters
        string = re.sub(r'\W', ' ', str(string))
        # remove all single characters
        string = re.sub(r'\s+[a-zA-Z]\s+', ' ', string)
        # Remove single characters from the start
        string = re.sub(r'\^[a-zA-Z]\s+', ' ', string)
        # Substituting multiple spaces with single space
        string = re.sub(r'\s+', ' ', string, flags=re.I)
        # Removing prefixed 'b'
        string = re.sub(r'^b\s+', '', string)
        # Converting to Lowercase
        string = string.lower()
        if full_process:
            # Lemmatization
            tokens = string.split()
            tokens = [stemmer.lemmatize(word) for word in tokens]
            tokens = [word for word in tokens if word not in en_stop]
            tokens = [word for word in tokens if len(word) > 3]
            string = ' '.join(tokens)
        return string

[nltk_data] Downloading package stopwords to /Users/kevin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [434]:
# Only needed if you want to see whats happening 
# def show_phrases(tokenized_sentences, threshold=10, shown=15):
#   '''Show collocation for a corpus of tokenized sentences.'''
#   phrases = Phrases(tokenized_sentences, threshold=threshold)
#   i = 0
#   for phrase, score in phrases.export_phrases().items():
#   #for phrase, score in phrases.export_phrases(): # for other gensim version 
#     if i>shown:
#       break
#     else:
#       print("Expression : {0}, score = {1}".format(phrase, score))
#     i=i+1

In [435]:
# Run me 
# this function hasn't been called yet 
def tokenize_pad_numericalize(entry, vocab_stoi, tok, pad=True, max_length=100):
    if pad :
        text = [ vocab_stoi[token] if token in vocab_stoi else vocab_stoi[''] for token in tok.tokenize(entry)]
        padded_text = None
        l = len(text)
        if l < max_length:   padded_text = text + [ vocab_stoi[''] for i in range(len(text), max_length) ] 
        elif l > max_length: padded_text = text[:max_length]
        else:                        padded_text = text
        return padded_text
    else : 
        text = [ vocab_stoi[token] if token in vocab_stoi else vocab_stoi[''] for token in tok.tokenize(entry)]
        return text

# this part gets ran to get the words that are commonly seen together 
pretrained_vectors = FastText(language='en')
pretrained_vocab = vocab(pretrained_vectors.stoi, min_freq=0)

unk_token = ""
unk_index = 0
pad_token = ''
pad_index = 1
pretrained_vocab.insert_token("",unk_index)
#pretrained_vocab.insert_token("",pad_index)

pretrained_vocab.set_default_index(unk_index)
pretrained_embeddings = pretrained_vectors.vectors


pretrained_embeddings = torch.cat((torch.zeros(2,pretrained_embeddings.shape[1]),pretrained_embeddings))
stoi = pretrained_vocab.get_stoi()
tok = TreebankWordTokenizer()

all_sentences = df[['pros','cons']].applymap(lambda x: nltk.word_tokenize(preprocess_transformers(x, full_process=False)))
all_sentences = all_sentences.melt().value.to_list()

phrases = Phrases(all_sentences, delimiter='oo' ,threshold=100)
phraser = Phraser(phrases)


In [86]:
# all_sentences 

[['very',
  'friendly',
  'and',
  'welcoming',
  'to',
  'new',
  'staff',
  'easy',
  'going',
  'ethic'],
 ['friendly', 'helpful', 'and', 'hard', 'working', 'colleagues'],
 ['easy',
  'to',
  'get',
  'the',
  'job',
  'even',
  'without',
  'experience',
  'in',
  'finance'],
 ['nice', 'staff', 'to', 'work', 'with'],
 ['easy', 'to', 'get', 'the', 'job', 'nice', 'colleagues'],
 ['some',
  'good',
  'people',
  'to',
  'work',
  'with',
  'flexible',
  'working',
  'out',
  'of',
  'hours',
  'language',
  'classes',
  'and',
  'aerobics',
  'financial',
  'support',
  'for',
  'exams'],
 ['good',
  'investment',
  'management',
  'strategy',
  'overall',
  'there',
  'are',
  'individual',
  'admin',
  'staff',
  'that',
  'are',
  'very',
  'nice',
  'and',
  'helpful'],
 ['the',
  'people',
  'are',
  'great',
  'and',
  'the',
  'culture',
  'is',
  'very',
  'forward',
  'thinking',
  'lots',
  'of',
  'development',
  'opportunities',
  'and',
  'really',
  'invest',
  'in',
  

In [436]:
# saved_phrases = show_phrases(all_sentences,threshold=100)

Expression : forward_thinking, score = 151.91193453916839
Expression : gym_membership, score = 345.9842278195271
Expression : pilates_classes, score = 144.2803157626687
Expression : co_workers, score = 126.23831271235483
Expression : answering_questions, score = 136.09277566144985
Expression : fast_paced, score = 109.01851668453199
Expression : dull_moment, score = 145.8368220417218
Expression : christmas_party, score = 104.70785424195036
Expression : mis_sold, score = 206.2029662647178
Expression : micro_managing, score = 106.76965568897316
Expression : aj_bell, score = 4549.543711733414
Expression : tunbridge_wells, score = 28615.59595959596
Expression : acca_qualification, score = 330.195347102429
Expression : guiding_principles, score = 172.95140415140415
Expression : bulge_bracket, score = 9298.854102999434
Expression : bogged_down, score = 156.12822519928935


In [437]:
test = False
if test:
    tokenizer = AutoTokenizer.from_pretrained('prajjwal1/bert-tiny')
else:
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [438]:
# Data maker for the transfomer
def make_inputs(df,feature):
    minutes_transformers = df[['date_review',feature]].set_index('date_review').copy()
    # standardize text
    minutes_transformers['sentence_treated'] = minutes_transformers[feature].apply(lambda x: preprocess_transformers(x, full_process=False))
    # Apply phraser
    minutes_transformers['sentence_treated'] = minutes_transformers['sentence_treated'].apply(lambda x: ' '.join(phraser[nltk.word_tokenize(x)]))
    # Set max length
    max_len = minutes_transformers['sentence_treated'].apply(lambda x:len(nltk.word_tokenize(x))).quantile(0.95)
    max_len = np.min((max_len,200)).astype(int)

    encoded_corpus = tokenizer(text=minutes_transformers['sentence_treated'].tolist(),
                            add_special_tokens=True,
                            padding='max_length',
                            truncation='longest_first',
                            max_length=max_len,
                            return_attention_mask=True)
                            
    print('The maximum sentence length for the transformers input is', max_len)
    input_ids      = encoded_corpus['input_ids']
    attention_mask = encoded_corpus['attention_mask']
    return input_ids, attention_mask

# pros and cons to feed to the transformer
input_pros, mask_pros = make_inputs(df,'pros')
input_cons, mask_cons = make_inputs(df,'cons')


The maximum sentence length for the transformers input is 41


KeyboardInterrupt: 

In [ ]:
# tt['sentence_treated'] = tt['pros'].apply(lambda x: preprocess_transformers(x, full_process=False))
# tt['sentence_treated'] = tt['sentence_treated'].apply(lambda x: ' '.join(phraser[nltk.word_tokenize(x)]))


In [93]:
# max_len = tt['sentence_treated'].apply(lambda x:len(nltk.word_tokenize(x))).quantile(0.95)
# max_len = np.min((max_len,200)).astype(int)

In [96]:
# encoded_corpus = tokenizer(text=tt['sentence_treated'].tolist(),
#                         add_special_tokens=True,
#                         padding='max_length',
#                         truncation='longest_first',
#                         max_length=max_len,
#                         return_attention_mask=True)

# input_ids      = encoded_corpus['input_ids']
# attention_mask = encoded_corpus['attention_mask']

In [ ]:
inputs = input_pros, input_cons, mask_pros, mask_cons

In [77]:
labels = df.work_life_balance.astype(int).to_numpy() - 1

In [17]:
# # Paramters for data loaders
# test_size  = 0
# seed       = 42
# batch_size = len(df)

# # Make splits for the different sets
# #train_index, test_index = train_test_split(range(0,len(labels)), test_size=test_size,random_state=seed, stratify=labels)
# #valid_index, test_index = train_test_split(test_index, test_size=0.5,random_state=seed, stratify=labels[test_index])
# data_index = [i for i in range(0, len(df)-1)]
# len(data_index)

838565

In [54]:
# Dataloaders with mask tensors
# def create_dataloaders(inputs, split_index, labels, batch_size):
#     tensor_input  = tuple(torch.tensor(input_)[split_index] for input_ in inputs)
#     labels_tensor = torch.tensor(labels)[split_index]
#     dataset    = TensorDataset(*tensor_input, labels_tensor)
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
#     return dataloader
 
# Create train/valid/test sets
# train_dataloader = create_dataloaders(inputs, data_index ,labels, batch_size)

In [55]:
# for i in range(3):
#     sample_pros, sample_cons, mask_pros, mask_cons, label_sample = next(iter(train_dataloader))

In [138]:
# df['tensor_headline'] = torch.Tensor(df['tokenized_headline'])

In [ ]:

# df['tokenized_headline'] = df.apply(lambda row: torch.Tensor(list(row["tokenized_headline"])).int(), axis=1)
# df['tokenized_headline'] = df.apply(lambda row: tokenizer.decode(row['tokenized_headline']), axis=1)

# df['tokenized_pros'] = df.apply(lambda row: torch.Tensor(list(row["tokenized_pros"])).int(), axis=1)
# df['tokenized_pros'] = df.apply(lambda row: tokenizer.decode(row['tokenized_pros']), axis=1)

# df['tokenized_cons'] = df.apply(lambda row: torch.Tensor(list(row["tokenized_cons"])).int(), axis=1)
# df['tokenized_cons'] = df.apply(lambda row: tokenizer.decode(row['tokenized_cons']), axis=1)


In [216]:
# tokenizer.decode(sample_pros[5])

'[CLS] some good people tooowork with flexible working out of hours language classes and aerobics financial support for exams [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [218]:
# type(sample_pros[5])

torch.Tensor

In [57]:
# for i in range(1,10):
#     print(i)
#     id_ = i
#     print('\nLabel:',label_sample[id_].numpy()
#         ,'\nSentence Pros:',tokenizer.decode(sample_pros[id_])
#         ,'\nPadding:',mask_pros[id_].numpy()
#         ,'\nSentence Cons:',tokenizer.decode(sample_cons[id_])
#         ,'\nPadding:',mask_cons[id_].numpy(),)

1

Label: 1 
Sentence Pros: [CLS] friendly helpful and hard working colleagues [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
Padding: [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0] 
Sentence Cons: [CLS] pooroosalary which doesn improve much with progression no incentive tooowork harder high turnover of staff pooroosystems [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] 
Padding: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2

Label: 0 
Sentence Pros: [CLS] easy to get the job even without experience in finance

In [273]:
# mask_cons[1]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [439]:
# Run me
#We use the same function for transformers and for LSTMs
df['headline'] = df['headline'].progress_apply(lambda x : preprocess_transformers(x, full_process=False))
df['pros'] = df['pros'].progress_apply(lambda x : preprocess_transformers(x, full_process=False))
df['cons'] = df['cons'].progress_apply(lambda x : preprocess_transformers(x, full_process=False))

df['headline'] = df['headline'].progress_apply(lambda x: ' '.join(phraser[word_tokenize(x)]))
df['pros'] = df['pros'].progress_apply(lambda x: ' '.join(phraser[word_tokenize(x)]))
df['cons'] = df['cons'].progress_apply(lambda x: ' '.join(phraser[word_tokenize(x)]))

100%|████████████████████████████████| 838566/838566 [00:54<00:00, 15380.16it/s]


In [485]:
# Run me
pad = True
# We use different lengths for each field, because the headlines are usually much shorter than the pros and cons
df['tokenized_headline'] = df['headline'].progress_apply(lambda x : tokenize_pad_numericalize(x, stoi, tok, pad, max_length=20))
df['tokenized_pros'] = df['pros'].progress_apply(lambda x : tokenize_pad_numericalize(x, stoi, tok, pad, max_length=80))
df['tokenized_cons'] = df['cons'].progress_apply(lambda x : tokenize_pad_numericalize(x, stoi, tok, pad, max_length=100))

100%|████████████████████████████████| 838566/838566 [00:38<00:00, 21762.62it/s]


In [89]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 300

In [287]:
# print(df.iloc[1, 16])

associated kilometres [unused7] ο [unused659] relationships [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [291]:
# df.head(1)

,firm,date_review,job_title,current,location,overall_rating,work_life_balance,culture_values,diversity_inclusion,career_opp,comp_benefits,senior_mgmt,recommend,ceo_approv,outlook,tokenized_headline,tokenized_pros,tokenized_cons
0,AFH-Wealth-Management,2015-04-05,,Current Employee,NaN,2,4.0,3.0,NaN,2.0,3.0,3.0,x,o,r,[unused539] relationships ∨ oral [unused890] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD],[unused221] associated [unused7]rong [unused11] [unused46] ˤ good [unused662]crow [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD...,[PAD] [PAD] [unused7]2 [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA...


In [295]:
torch.Tensor(short.iloc[1, -4]).int()

tensor([3378, 3717,    8, 1169,  664, 6550,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0], dtype=torch.int32)

In [296]:
tokenizer.decode(torch.Tensor(short.iloc[1, -4]).int())

'associated kilometres [unused7] ο [unused659] relationships [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [481]:
def imtrying(row1, row2):
    list_tokens = []
    if len(row1.split()) < len(row2):
        for idx, val in enumerate(row1.split()):
            if row2[idx] > 1000:
                list_tokens.append(val)
    else:
        for idx, val in enumerate(row2):
            if val > 1000:
                list_tokens.append(row1[idx])
    return " ".join(list_tokens)

In [476]:
# test function 
s = "very friendly and welcoming to new staff easy going ethic"
s2 = [226, 3378, 8, 17583, 12, 47, 1154, 2204, 667, 24375, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [480]:
imtrying(s, s2)

'friendly welcoming staff easy ethic'

In [486]:
df['tokenized_headline']=df.apply(lambda row: imtrying(row['headline'], row['tokenized_headline']), axis=1)
df['tokenized_pros']=df.apply(lambda row: imtrying(row['pros'], row['tokenized_pros']), axis=1)
df['tokenized_cons']=df.apply(lambda row: imtrying(row['cons'], row['tokenized_cons']), axis=1)

In [488]:
df.drop(['pros','cons','headline'], axis=1, inplace=True)

In [491]:
df.to_csv("tokenized_data2", sep='\t', encoding='utf-8')